<a href="https://colab.research.google.com/github/AlfredoMarino/easy-money/blob/main/notebooks/3.0-aamv-preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocesado de datos

In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import KNNImputer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
__DATA_PATH__ = "/content/drive/MyDrive/colab_notebooks/easy-money/data"
__RAW_DATA_PATH__ = f"{__DATA_PATH__}/raw"
__INTERIM_DATA_PATH__ = f"{__DATA_PATH__}/interim"

__OUTPUT_DF_NAME__ = "easy_money_prep_1"

__PREDICT_SALARY__ = False
__DO_ONE_HOT_ENCODE__ = True


### FUNCTIONS

In [4]:
def one_hot_encode(dataframe: pd.DataFrame, column_name: str) -> pd.DataFrame:
    _dummy_dataset = pd.get_dummies(dataframe[column_name], prefix=column_name)
    return pd.concat([dataframe, _dummy_dataset], axis=1).drop([column_name], axis=1)


def encode_dataframe(dataframe: pd.DataFrame) -> pd.DataFrame:
    cat_columns = [col for col in dataframe.select_dtypes(exclude=[np.number, "datetime"]).columns]
    print(f"columns to encode: {cat_columns}")

    print(f"dataframe before one-hot encoding: {dataframe.shape}")
    for cat_col in cat_columns:
        dataframe = one_hot_encode(dataframe, cat_col)

    print(f"dataframe after one-hot encoding:  {dataframe.shape}")
    return dataframe


def export_to_csv(dataframe: pd.DataFrame, filename: str, path: str = __INTERIM_DATA_PATH__):
    exported_path = f"{path}/{filename}.csv"
    dataframe.to_csv(exported_path)
    print(f"Dataframe exported to {exported_path}")


def predict_salaries(dataframe: pd.DataFrame) -> pd.DataFrame:
    """
    predict the salary of clients who have NAN salaries using KNNImputer,
    based on the closest values as a function of age
    :param dataframe: dataframe to used
    :return: salary_df
    """
    _salary = dataframe.copy(deep=True).groupby("pk_cid").agg(
        age=("age", np.max),
        salary=("salary", np.mean),
    ).sort_values("age")
    _salary.reset_index(inplace=True)

    columns_to_evaluate = ["age", "salary"]

    is_null_condition = _salary["salary"].isnull()
    is_outlier_condition = _salary["salary"] <= np.quantile(_salary[_salary["salary"].notnull()]["salary"], q=0.99)
    salary_train_df = _salary[is_null_condition | is_outlier_condition]

    print("Predicting salary...")
    imputer = KNNImputer(n_neighbors=2)
    imputer.fit(salary_train_df[columns_to_evaluate])
    prediction = imputer.transform(_salary[columns_to_evaluate])
    salary_predict = pd.DataFrame(prediction, columns=["age", "salary_prediction"])

    _salary["knn_salary"] = salary_predict["salary_prediction"]
    return _salary


def input_salaries(dataframe: pd.DataFrame, salary_df: pd.DataFrame) -> pd.DataFrame:
    salaries_reshaped = pd.merge(
        left=dataframe,
        right=salary_df[["pk_cid", "knn_salary"]],
        on="pk_cid",
        how="left"
    )
    print(f"salary_df.shape: {salary_df.shape}")
    print(f"dataframe.shape: {dataframe.shape}")
    print(f"salaries_reshaped.shape: {salaries_reshaped.shape}")

    salaries_reshaped.drop("salary", axis=1, inplace=True)
    salaries_reshaped.rename(columns={"knn_salary": "salary"}, inplace=True)
    return salaries_reshaped


def set_other(dataframe, column, num_values):
    top_values = dataframe[column].value_counts().head(num_values)
    top_values_list = top_values.index.to_list()
    return set_other_by_top_values(dataframe, column, top_values_list)


def set_other_by_top_values(dataframe, column, top_values_list):
    top_values_list.append('OTHERS')
    _df = dataframe.copy(deep=True)
    _df[column] = pd.Categorical(dataframe[column], categories=top_values_list)
    return _df[column].fillna('OTHERS')

## Import data

In [5]:
# Load products_df
products_df = pd.read_csv(f"{__RAW_DATA_PATH__}/products_df.csv")
del (products_df["Unnamed: 0"])
print(f"* products_df loaded           *  shape: {products_df.shape}")

# Load commercial_activity_df
commercial_activity_df = pd.read_csv(f"{__RAW_DATA_PATH__}/commercial_activity_df.csv")
del (commercial_activity_df["Unnamed: 0"])
print(f"* commercial_activity_dfloaded *  shape: {commercial_activity_df.shape}")

# Load sociodemographic_df
sociodemographic_df = pd.read_csv(f"{__RAW_DATA_PATH__}/sociodemographic_df.csv")
del (sociodemographic_df["Unnamed: 0"])
print(f"* sociodemographic_df loaded   *  shape: {sociodemographic_df.shape}")

* products_df loaded           *  shape: (5962924, 17)
* commercial_activity_dfloaded *  shape: (5962924, 6)
* sociodemographic_df loaded   *  shape: (5962924, 8)


## Merge Dataframes

In [6]:
_df = pd.merge(left=commercial_activity_df, right=sociodemographic_df, how="inner", on=["pk_cid", "pk_partition"])
df = pd.merge(left=_df, right=products_df, how="inner", on=["pk_cid", "pk_partition"])
print(f"* Dataframes merged *  shape: {df.shape}")

* Dataframes merged *  shape: (5962924, 27)


## Data Preprocessing

In [7]:
df_prep = df.copy(deep=True)

##### pk_partition

In [8]:
df_prep["pk_partition"] = pd.to_datetime(df_prep["pk_partition"])

##### entry_date

In [9]:
# Obteníamos un error al convertir nuestra variable a fecha debido a las fechas correspondientes
# al 29 de Febrero de los años 2015 y 2019 puesto que no existen.
# Por este motivo, decidimos reemplazar estos valores por la fecha más cercana (día anterior)
df_prep["entry_date"] = df_prep["entry_date"].replace("2015-02-29", "2015-02-28")
df_prep["entry_date"] = df_prep["entry_date"].replace("2019-02-29", "2019-02-28")

df_prep["entry_date"] = pd.to_datetime(df_prep["entry_date"])

#Generamos atributos extras a partir de entry_date
df_prep["entry_date_month"] = df_prep["entry_date"].dt.month.astype(int)
df_prep["entry_date_year"] = df_prep["entry_date"].dt.year.astype(int)
df_prep["entry_date_day"] = df_prep["entry_date"].dt.day.astype(int)
df_prep["entry_date_week"] = df_prep["entry_date"].dt.isocalendar().week.astype(int)
df_prep["entry_date_weekday"] = df_prep["entry_date"].dt.weekday.astype(int)
df_prep["entry_date_trim"] = df_prep["entry_date"].dt.quarter.astype(int)

##### entry_channel

In [10]:
df_prep["entry_channel"].fillna("UNKNOWN", inplace=True)

In [11]:
entry_channel_df = df_prep.groupby("pk_cid").agg(
    entry_channel=("entry_channel", lambda e: e.mode().iat[0])
)
top2 = entry_channel_df.value_counts().head(2).reset_index()["entry_channel"].to_list()
top2

['KHE', 'KHQ']

In [12]:
entry_channel_df.value_counts()

entry_channel
KHE              196615
KHQ               76137
KFC               60175
KHM               32114
KAT               27008
                  ...  
KBU                   1
KBH                   1
KBE                   1
KAI                   1
KCL                   1
Length: 68, dtype: int64

In [13]:
df_prep["entry_channel"] = set_other_by_top_values(df_prep, "entry_channel", top2)
df_prep["entry_channel"].value_counts(dropna=False)

KHE       3113947
OTHERS    2258697
KHQ        590280
Name: entry_channel, dtype: int64

##### active_customer

In [14]:
df_prep["active_customer"] = df_prep["active_customer"].astype(int)

##### region_code

In [15]:
df_prep["region_code"] = df_prep["region_code"].fillna(-1).astype(int)
df_prep["region_code"].value_counts(dropna=False)

 28    1185757
 8      600362
 46     358142
 30     296856
 41     290447
 15     254891
 29     187911
 36     175850
 3      171358
 11     160894
 33     134562
 50     132983
 6      122587
 35     121982
 47     116948
 18     102789
 45     102328
 37      92538
 10      83283
 14      75405
 2       72071
 21      72037
 13      70092
 39      69589
 12      68709
 7       62436
 32      54639
 27      52783
 43      51782
 17      50749
 9       47746
 25      46887
 16      36094
 24      35426
 48      35256
 26      34790
 4       32680
 31      31241
 49      30965
 23      30955
 38      29001
 34      27545
 19      26762
 40      21658
 5       21300
 22      21112
 20      18833
 1       12999
 44      12616
 42       8500
 51       2896
 52       2638
-1        2264
Name: region_code, dtype: int64

##### gender

In [16]:
df_prep.dropna(subset=["gender"], inplace=True)
df_prep["gender"].replace("H", 1, inplace=True)
df_prep["gender"].replace("V", 0, inplace=True)
df_prep["gender"].value_counts(dropna=False)

1    3087502
0    2875397
Name: gender, dtype: int64

##### deceased

In [17]:
df_prep["deceased"].replace("N", 0, inplace=True)
df_prep["deceased"].replace("S", 1, inplace=True)
df_prep["deceased"].value_counts()

0    5961824
1       1075
Name: deceased, dtype: int64

##### payroll and payroll_account

In [18]:
df_prep.dropna(subset=["payroll", "payroll_account"], inplace=True)
df_prep["payroll"] = df_prep["payroll"].astype(int)
df_prep["payroll"].value_counts(dropna=False)

0    5757051
1     205787
Name: payroll, dtype: int64

##### pension_plan

In [19]:
df_prep["pension_plan"] = df_prep["pension_plan"].astype(int)

##### salary

In [20]:
%%time
if __PREDICT_SALARY__:
    salary_df = predict_salaries(df_prep)
    salary_df.to_csv(f"{__INTERIM_DATA_PATH__}/salary_df.csv")
else:
    salary_df = pd.read_csv(f"{__INTERIM_DATA_PATH__}/salary_df.csv", index_col=0)

print(f"salary_df.shape: {salary_df.shape}")
salary_df.head(300)

salary_df.shape: (456371, 4)
CPU times: user 254 ms, sys: 78.6 ms, total: 333 ms
Wall time: 814 ms


In [21]:
df_prep = input_salaries(df_prep, salary_df)

salary_df.shape: (456371, 4)
dataframe.shape: (5962838, 33)
salaries_reshaped.shape: (5962838, 34)


##### segment

In [22]:
df_prep["segment"].fillna("00 - OTROS", inplace=True)
df_prep.groupby("segment").agg(
    salary=("salary", np.mean),
    count=("pk_cid", len)
).sort_values("salary")

,salary,count
segment,,
00 - OTROS,113472.225547,133928
02 - PARTICULARES,114974.497771,1830815
03 - UNIVERSITARIO,119872.217280,3900157
01 - TOP,139847.212802,97938


In [23]:
df_prep["segment"].replace("00 - OTROS", 0, inplace=True)
df_prep["segment"].replace("02 - PARTICULARES", 1, inplace=True)
df_prep["segment"].replace("03 - UNIVERSITARIO", 2, inplace=True)
df_prep["segment"].replace("01 - TOP", 3, inplace=True)
df_prep["segment"].value_counts()

2    3900157
1    1830815
0     133928
3      97938
Name: segment, dtype: int64

In [24]:
df_prep.isnull().sum()

pk_cid                0
pk_partition          0
entry_date            0
entry_channel         0
active_customer       0
segment               0
country_id            0
region_code           0
gender                0
age                   0
deceased              0
short_term_deposit    0
loans                 0
mortgage              0
funds                 0
securities            0
long_term_deposit     0
em_account_pp         0
credit_card           0
payroll               0
pension_plan          0
payroll_account       0
emc_account           0
debit_card            0
em_account_p          0
em_acount             0
entry_date_month      0
entry_date_year       0
entry_date_day        0
entry_date_week       0
entry_date_weekday    0
entry_date_trim       0
salary                0
dtype: int64

## Export Dataframe

In [25]:
df_prep_not_null = df_prep.copy(deep=True).sort_values(["pk_cid", "pk_partition"]).reset_index(drop=True)

In [ ]:
%%time
if __DO_ONE_HOT_ENCODE__:
    df_prep_one_hot = encode_dataframe(df_prep_not_null)
    export_to_csv(df_prep_one_hot, __OUTPUT_DF_NAME__)
else:
    export_to_csv(df_prep_not_null, __OUTPUT_DF_NAME__)

columns to encode: ['entry_channel', 'country_id']
dataframe before one-hot encoding: (5962838, 33)
dataframe after one-hot encoding:  (5962838, 75)
Dataframe exported to ../data/interim/easy_money_prep_1.csv
Wall time: 3min 5s
